In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.graph_objects as go

# Sample data creation
data = {
    'conversation_date': pd.date_range(start='2023-11-01', periods=121, freq='D'),
    'contacted': np.random.randint(100, 500, size=121)  # Random integers between 100 and 500
}

df = pd.DataFrame(data)
df['conversation_date'] = pd.to_datetime(df['conversation_date'])
df.set_index('conversation_date', inplace=True)

# Calculating the IQR for 'contacted' to filter outliers
Q1 = df['contacted'].quantile(0.25)
Q3 = df['contacted'].quantile(0.75)
IQR = Q3 - Q1
filter_lower = Q1 - 1.0 * IQR  # Tighter threshold
filter_upper = Q3 + 1.0 * IQR  # Tighter threshold

# Filtering outliers
filtered_df = df[(df['contacted'] > filter_lower) & (df['contacted'] < filter_upper)]

# Resampling data into weekly sums for 'contacted' after removing outliers
weekly_data_contacted = filtered_df['contacted'].resample('W-MON').sum()

# Setup Exponential Smoothing model for 'contacted'
# Remove trend component if it's not justified by data
exp_smoothing_model_contacted = ExponentialSmoothing(
    weekly_data_contacted,
    seasonal='add',
    seasonal_periods=4,
    trend=None,  # Consider removing or modifying the trend if not needed
    damped_trend=False
)

# Fit the model
exp_smoothing_results_contacted = exp_smoothing_model_contacted.fit()

# Forecasting the next 3 months on a weekly basis, which is approximately 12 weeks
forecast_values_contacted_es = exp_smoothing_results_contacted.forecast(steps=12)
forecast_index_contacted_es = pd.date_range(start=weekly_data_contacted.index[-1], periods=13, freq='W-MON')[1:]

# Plotting the actual vs forecasted data for 'contacted'
fig_contacted_es = go.Figure()
fig_contacted_es.add_trace(go.Scatter(
    x=weekly_data_contacted.index,
    y=weekly_data_contacted,
    mode='lines+markers',
    name='Actual Contacted - Weekly',
    line=dict(color='royalblue')
))

fig_contacted_es.add_trace(go.Scatter(
    x=forecast_index_contacted_es,
    y=forecast_values_contacted_es,
    mode='lines+markers',
    name='Forecasted Contacted - Exponential Smoothing Weekly',
    line=dict(color='firebrick')
))

fig_contacted_es.update_layout(
    title='Weekly Contacted Forecast - Exponential Smoothing Model After Removing Outliers',
    xaxis_title='Week',
    yaxis_title='Contacted',
    template='plotly_white'
)

fig_contacted_es.show()


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.graph_objects as go

# Generate sample data with a more realistic distribution centered around 140 seconds
np.random.seed(0)  # For reproducibility
data = {
    'conversation_date': pd.date_range(start='2023-11-01', periods=121, freq='D'),
    'avg_handle_time': np.random.normal(140, 20, size=121)  # Normal distribution around 140 seconds
}

df = pd.DataFrame(data)
df['conversation_date'] = pd.to_datetime(df['conversation_date'])
df.set_index('conversation_date', inplace=True)

# Calculating the IQR for 'avg_handle_time' to filter outliers
Q1 = df['avg_handle_time'].quantile(0.25)
Q3 = df['avg_handle_time'].quantile(0.75)
IQR = Q3 - Q1
filter_lower = Q1 - 1.5 * IQR
filter_upper = Q3 + 1.5 * IQR

# Filtering outliers
filtered_df = df[(df['avg_handle_time'] > filter_lower) & (df['avg_handle_time'] < filter_upper)]

# Resampling data into weekly averages for 'avg_handle_time' after removing outliers
weekly_avg_handle_time = filtered_df['avg_handle_time'].resample('W-MON').mean()

# Setup Exponential Smoothing model for 'avg_handle_time'
exp_smoothing_model_avg_time = ExponentialSmoothing(
    weekly_avg_handle_time,
    seasonal='add',
    seasonal_periods=4,
    trend='add',
    damped_trend=True
)

# Fit the model
exp_smoothing_results_avg_time = exp_smoothing_model_avg_time.fit()

# Forecasting the next 12 weeks
forecast_values_avg_time_es = exp_smoothing_results_avg_time.forecast(steps=12)
forecast_index_avg_time_es = pd.date_range(start=weekly_avg_handle_time.index[-1], periods=13, freq='W-MON')[1:]

# Plotting the actual vs forecasted data for 'avg_handle_time'
fig_avg_time_es = go.Figure()
fig_avg_time_es.add_trace(go.Scatter(
    x=weekly_avg_handle_time.index,
    y=weekly_avg_handle_time,
    mode='lines+markers',
    name='Actual Avg Handle Time - Weekly Average',
    line=dict(color='royalblue')
))

fig_avg_time_es.add_trace(go.Scatter(
    x=forecast_index_avg_time_es,
    y=forecast_values_avg_time_es,
    mode='lines+markers',
    name='Forecasted Avg Handle Time - Exponential Smoothing Weekly',
    line=dict(color='firebrick')
))

fig_avg_time_es.update_layout(
    title='Weekly Avg Handle Time Forecast - Exponential Smoothing Model After Adjusted Outlier Removal',
    xaxis_title='Week',
    yaxis_title='Avg Handle Time (seconds)',
    template='plotly_white'
)

fig_avg_time_es.show()
